# AUTONORMALIZE DEMO 
Using Autonormalize to normalize a kaggle dataset about liquor purchasing orders.

In [ ]:
normalize_dependencies(dependencies)

In [ ]:
split_dataframe(df, new_grps)

In [1]:
import os

import pandas as pd
import autonormalize as an

In [7]:
liquor_df = pd.read_csv(os.path.join(os.getcwd(), 'autonormalize/downloads/liquor.csv'))
liquor_df = liquor_df.drop(columns=liquor_df.columns[12:])
liquor_df = liquor_df.drop(range(3000000, 12591077))
liquor_df_smaller = liquor_df.drop(range(1500000, 3000000))
liquor_df = liquor_df.dropna()
liquor_df = liquor_df.drop_duplicates()
liquor_df_smaller = liquor_df_smaller.dropna()
liquor_df_smaller = liquor_df_smaller.drop_duplicates()
liquor_df.head(3)

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,Category,Category Name
6,S28865700001,11/09/2015,2538,Hy-Vee Food Store #3 / Waterloo,1422 FLAMMANG DR,WATERLOO,50702,"1422 FLAMMANG DR\nWATERLOO 50702\n(42.459938, ...",7.0,Black Hawk,1701100.0,DECANTERS & SPECIALTY PACKAGES
8,S29339300091,11/30/2015,2662,Hy-Vee Wine & Spirits / Muscatine,"522 MULBERRY, SUITE A",MUSCATINE,52761,"522 MULBERRY, SUITE A\nMUSCATINE 52761\n",70.0,Muscatine,1701100.0,DECANTERS & SPECIALTY PACKAGES
13,S28866900001,11/11/2015,3650,"Spirits, Stogies and Stuff",118 South Main St.,HOLSTEIN,51025,118 South Main St.\nHOLSTEIN 51025\n(42.490073...,47.0,Ida,1701100.0,DECANTERS & SPECIALTY PACKAGES


In [4]:
deps_exact = an.find_dependencies(liquor_df, 1.00)
print(deps_exact)

100%|██████████| 11/11 [08:54<00:00, 38.89s/it]

 --> Invoice/Item Number
 {Invoice/Item Number}  --> Date
 {Zip Code,Date,Address}  {Date,Address,Store Name}  {Zip Code,Date,Store Name}  {Date,Store Location}  {Invoice/Item Number}  --> Store Number
 {Store Number}  {Zip Code,Date,Address}  {Invoice/Item Number}  {Date,Store Location}  --> Store Name
 {Zip Code,Date,Store Name}  {Store Number}  {Store Name,Store Location}  {Invoice/Item Number}  --> Address
 {Zip Code,Date,Address}  {Store Location}  {County Number,Address,Date}  {Zip Code,Date,Store Name}  {County,Date,Address}  {Store Number}  {Store Name,Address}  {Invoice/Item Number}  --> City
 {Store Number}  {Store Name,Address}  {Store Location}  {Invoice/Item Number}  --> Zip Code
 {Store Number}  {Zip Code,Date,Address}  {Invoice/Item Number}  {Zip Code,City,Address}  --> Store Location
 {County}  {Store Location}  {Zip Code,Address}  {Store Number}  {Store Name,Address}  {Invoice/Item Number}  --> County Number
 {Store Location}  {Zip Code,Address}  {Store Number}  {Count

In [8]:
deps_approx = an.find_dependencies(liquor_df_smaller, 0.96)
print(deps_approx)

100%|██████████| 11/11 [53:58<00:00, 132.91s/it]  


 --> Invoice/Item Number
 {Invoice/Item Number}  --> Date
 {Invoice/Item Number}  {Store Location}  {Store Name}  {Address}  --> Store Number
 {Invoice/Item Number}  {Store Location}  {Store Number}  {Address}  --> Store Name
 {Invoice/Item Number}  {Store Location}  {Store Name}  {Store Number}  --> Address
 {Store Name}  {Store Location}  {Store Number}  {Zip Code}  {Invoice/Item Number}  {Address}  --> City
 {Store Name}  {Store Location}  {Store Number}  {Invoice/Item Number}  {Address}  --> Zip Code
 {Invoice/Item Number}  {Address}  {Store Name}  {Store Number}  --> Store Location
 {City}  {County}  {Store Name}  {Store Location}  {Store Number}  {Zip Code}  {Invoice/Item Number}  {Address}  --> County Number
 {City}  {Store Name}  {Store Location}  {County Number}  {Store Number}  {Zip Code}  {Invoice/Item Number}  {Address}  --> County
 {Invoice/Item Number}  {Category Name}  --> Category
 {Invoice/Item Number}  {Category}  --> Category Name


In [9]:
groupings = an.normalize_dependencies(deps_approx)
for grp in groupings:
    print('\n~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~\n')
    print(grp)


~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~

 --> Invoice/Item Number
 {Invoice/Item Number}  --> Date
 {Invoice/Item Number}  --> Store Name
 {Invoice/Item Number}  --> Category Name

~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~

 {Category}  --> Category Name
 {Category Name}  --> Category

~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~

 {Store Location}  {Store Number}  {Address}  --> Store Name
 {Store Location}  {Store Name}  {Address}  --> Store Number
 {Store Location}  {Store Name}  {Store Number}  --> Address
 {Store Name}  {Store Location}  {Store Number}  {Address}  --> Zip Code
 {Address}  {Store Name}  {Store Number}  --> Store Location

~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~

 --> Zip Code
 {Zip Code}  --> City

~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~

 --> City
 {City}  --> County

~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~

 {County Number}  --> County
 {County}  --> County Number


In [10]:
new_dfs = an.split_dataframe(liquor_df_smaller, groupings)
new_dfs

[        Invoice/Item Number        Date                   Store Name  \
 0              306831300015  07/30/2012    Bootleggin' Barzini's Fin   
 1              306831300016  07/30/2012    Bootleggin' Barzini's Fin   
 2              306831300020  07/30/2012    Bootleggin' Barzini's Fin   
 3                 378900005  10/17/2012    Wal-Mart 1393 / Oskaloosa   
 4                 378900006  10/17/2012    Wal-Mart 1393 / Oskaloosa   
 5                 378900013  10/17/2012    Wal-Mart 1393 / Oskaloosa   
 6                 378900015  10/17/2012    Wal-Mart 1393 / Oskaloosa   
 7                 378900020  10/17/2012    Wal-Mart 1393 / Oskaloosa   
 8                 378900029  10/17/2012    Wal-Mart 1393 / Oskaloosa   
 9                 378900031  10/17/2012    Wal-Mart 1393 / Oskaloosa   
 10                381800014  02/29/2012          Round Window Liquor   
 11                381800016  02/29/2012          Round Window Liquor   
 12                381800019  02/29/2012          R

In [18]:
len(new_dfs)

6

In [12]:
new_dfs[0]

,Invoice/Item Number,Date,Store Name,Category Name
0,306831300015,07/30/2012,Bootleggin' Barzini's Fin,VODKA FLAVORED
1,306831300016,07/30/2012,Bootleggin' Barzini's Fin,VODKA FLAVORED
2,306831300020,07/30/2012,Bootleggin' Barzini's Fin,VODKA 80 PROOF
3,378900005,10/17/2012,Wal-Mart 1393 / Oskaloosa,STRAIGHT RYE WHISKIES
4,378900006,10/17/2012,Wal-Mart 1393 / Oskaloosa,BLENDED WHISKIES
5,378900013,10/17/2012,Wal-Mart 1393 / Oskaloosa,VODKA 80 PROOF
6,378900015,10/17/2012,Wal-Mart 1393 / Oskaloosa,VODKA 80 PROOF
7,378900020,10/17/2012,Wal-Mart 1393 / Oskaloosa,SPICED RUM
8,378900029,10/17/2012,Wal-Mart 1393 / Oskaloosa,AMERICAN COCKTAILS
9,378900031,10/17/2012,Wal-Mart 1393 / Oskaloosa,TRIPLE SEC


In [13]:
new_dfs[1]

,Category,Category Name
0,1011100.0,BLENDED WHISKIES
1,1011200.0,STRAIGHT BOURBON WHISKIES
2,1011250.0,SINGLE BARREL BOURBON WHISKIES
3,1011300.0,TENNESSEE WHISKIES
4,1011400.0,BOTTLED IN BOND BOURBON
5,1011500.0,STRAIGHT RYE WHISKIES
6,1011600.0,CORN WHISKIES
7,1012100.0,CANADIAN WHISKIES
8,1012200.0,SCOTCH WHISKIES
9,1012210.0,SINGLE MALT SCOTCH


In [14]:
new_dfs[2]

,Store Number,Store Name,Address,Zip Code,Store Location
0,4645,Kwik Shop #589 / Eldridge,"1, GROVE RD",52748,"1, GROVE RD\nELDRIDGE 52748\n(41.694958, -90.5..."
1,3576,Liquor Store / Hartley,10 3RD ST NW,51346,"10 3RD ST NW\nHARTLEY 51346\n(43.184821, -95.4..."
2,2548,Hy-Vee Food Store / Altoona,100 8TH STREET SW,50009,"100 8TH STREET SW\nALTOONA 50009\n(41.644041, ..."
3,2132,Spirits and Such,100 E ELM,52175,"100 E ELM\nWEST UNION 52175\n(42.96126, -91.80..."
4,4681,Spirits & Such,100 E ELM ST,52175,"100 E ELM ST\nWEST UNION 52175\n(42.961269, -9..."
5,5196,The Depot Coralville,100 E OAKDALE BLVD,52241,100 E OAKDALE BLVD\nCORALVILLE 52241\n(41.7015...
6,4140,Fareway Stores #900 / Euclid,100 EUCLID AVE,50313,"100 EUCLID AVE\nDES MOINES 50313\n(41.627684, ..."
7,4769,Elma Locker and Grocery,100 GRANT ST,50628,100 GRANT ST\nELMA 50628\n
8,4703,Walgreens #00910 / Sioux City,100 PIERCE ST,51101,"100 PIERCE ST\nSIOUX CITY 51101\n(42.490838, -..."
9,3688,Fredericksburg Food Center,100 W MAIN ST,50630,100 W MAIN ST\nFREDERICKSBURG 50630\n(42.96509...


In [15]:
new_dfs[3]

,City,Zip Code
0,ADAIR,50002
1,ADEL,50003
2,ALDEN,50006
3,ALTOONA,50009
4,AMES,50010
5,AMES,50014
6,AMES,50015
7,ANITA,50020
8,ANKENY,50021
9,ATLANTIC,50022


In [16]:
new_dfs[4]

,City,County
0,ACKLEY,Webster
1,ADAIR,Adair
2,ADEL,Dallas
3,AFTON,Union
4,AKRON,Plymouth
5,ALBIA,Monroe
6,ALDEN,Hardin
7,ALGONA,Kossuth
8,ALLISON,Butler
9,ALTA,Buena Vista


In [17]:
new_dfs[5]

,County Number,County
0,1.0,Adair
1,2.0,Adams
2,3.0,Allamakee
3,4.0,Appanoose
4,5.0,Audubon
5,6.0,Benton
6,7.0,Black Hawk
7,8.0,Boone
8,9.0,Bremer
9,10.0,Buchanan
